<a href="https://colab.research.google.com/github/01460421/01460421.github.io/blob/main/%E3%80%8Cpjsk_%E5%88%86%E6%95%B8%E7%A8%8B%E5%BC%8F%E5%84%AA%E5%8C%96%E5%BE%8C%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#先加載數據 其他人綜合力默認跟自己一樣

#先點這個加載數據

In [ ]:
import requests
import pandas as pd
import math

MUSIC_API_URL = 'https://raw.githubusercontent.com/Sekai-World/sekai-master-db-diff/refs/heads/main/musics.json'
# META_API_URL = 'https://raw.githubusercontent.com/RilakKuma2/prsk-calc/refs/heads/main/src/data/music_metas.json'
DIFF_API = 'https://raw.githubusercontent.com/Sekai-World/sekai-master-db-diff/refs/heads/main/musicDifficulties.json'
META_API_URL = 'https://sekai-data.3-3.dev/music_metas.json'

def get_music_titles():
    try:
        resp = requests.get(MUSIC_API_URL)
        resp.raise_for_status()
        return {item['id']: item['title'] for item in resp.json()}
    except Exception as e:
        print(f"無法獲取歌曲清單: {e}")
        return {}

def fetch_json(url):
    try:
        r = requests.get(url)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        print(f"抓取失敗 ({url}): {e}")
        return None


titles_map = get_music_titles()
resp = requests.get(META_API_URL)
diff = fetch_json(DIFF_API)

#預設綜合力30萬 技能3.0 s6未設定 隊友都一樣 預設活動加成100%

In [ ]:
import requests
import pandas as pd
import math

MUSIC_API_URL = 'https://raw.githubusercontent.com/Sekai-World/sekai-master-db-diff/refs/heads/main/musics.json'
META_API_URL = 'https://raw.githubusercontent.com/RilakKuma2/prsk-calc/refs/heads/main/src/data/music_metas.json'

USER_POWER = 300000
SKILL_MAGNIFICATION = 2

DIFFICULTY_RANK = {
    'easy': 1, 'normal': 2, 'hard': 3,
    'expert': 4, 'master': 5, 'append': 6
}


def calculate_event_pt(meta, power, skill_mag):
    eff_multiplier = (
        (meta['base_score'] * 1) +
        (meta['fever_score'] * 0.5) +
        (sum(meta['skill_score_multi']) * skill_mag)
    )
    score_part = math.floor((eff_multiplier * power * 4 + 0.075 * power * 5) / 17000)

    event_pt = math.floor((score_part + 123) * (meta['event_rate'] / 100) * 2)

    return event_pt

def calculate_efficiency():
    metas = resp.json()

    metas.sort(key=lambda x: (x['music_id'], DIFFICULTY_RANK.get(x['difficulty'], 0)))

    scored_list = []
    for m in metas:
        music_id = m['music_id']
        if music_id not in titles_map:
            continue
        event_pt = calculate_event_pt(m, USER_POWER, SKILL_MAGNIFICATION)

        scored_list.append({
            '歌曲標題': titles_map[music_id],
            '難度': m['difficulty'],
            '時間': m['music_time'],
            '活動PT': event_pt
        })

    scored_list.sort(key=lambda x: x['活動PT'], reverse=True)

    efficiency_result = []
    min_duration = float('inf')

    for song in scored_list:
        if song['時間'] < min_duration:
            efficiency_result.append(song)
            min_duration = song['時間']

    return pd.DataFrame(efficiency_result)

if __name__ == "__main__":
    df_efficiency = calculate_efficiency()

    print("\n世界計畫活動效率曲清單 (Pareto Frontier)：\n")
    display(df_efficiency)

#自己設定 技能打實效別打倍率 e.g.:3.7 -> 2.7

相對效率曲


In [ ]:
# --- 設定與常數 ---
USER_POWER = int(input("綜合力"))
SKILL_MAGNIFICATION = round(sum(map(float,input("技能以空白鍵分開").split(" "))) / 5,2)
s6 = float(input("s6"))
maxdiff = int(input("最高難度(lv)"))
bonus = float(input("活動加成")) / 100
times = 3600 / float(input("周回")) - 74.8
# energy_per_hour = float(input("每小時消耗體力"))
# no_append = int(input("append(0無1有)"))
# USER_POWER = 300000
# SKILL_MAGNIFICATION = 2.5
# s6 = 2.5
# maxdiff = 38
# bonus = 395 / 100
# times = 3600 / 28 - 74.8
energy_per_hour = 100
no_append = False

DIFFICULTY_RANK = {
    'easy': 1, 'normal': 2, 'hard': 3,
    'expert': 4, 'master': 5, 'append': 6
}

def calculate_event_pt(meta, power, skill_mag):
    eff_multiplier = (
        (meta['base_score'] * 1) +
        (meta['fever_score'] * 0.5) +
        (sum(meta['skill_score_multi'][0:5]) * (skill_mag) ) +
        (meta['skill_score_multi'][5] * s6)
    )

    score_part = math.floor((eff_multiplier * power * 4 + 0.075 * power * 5) / 17000)
    event_pt = math.floor((score_part + 123) * (meta['event_rate'] / 100) * (1 + bonus))

    return event_pt

def get_levels_map():

    if not diff: return {}
    return {(d['musicId'], d['musicDifficulty']): d['playLevel'] for d in diff}

def best_pt(song):
    use_energy = min(energy_per_hour,song['周回'] * 10)
    pt = 0

    if use_energy >= song['周回'] * 5:
        pt += song['活動PT'] * song['周回'] * 5 * 5
        use_energy -= song['周回'] * 5
        pt += song['活動PT'] * use_energy * 2
    else:
        pt += use_energy * song['活動PT'] * 5
        pt += song['活動PT'] * (song['周回'] - (use_energy / 5))

    return pt

def calculate_efficiency():
    levels = get_levels_map()
    metas = resp.json()

    metas.sort(key=lambda x: (x['music_id'], DIFFICULTY_RANK.get(x['difficulty'], 0)))

    scored_list = []
    for m in metas:
        music_id = m['music_id']
        diff = m['difficulty']
        if music_id not in titles_map:
            continue

        event_pt = calculate_event_pt(m, USER_POWER, SKILL_MAGNIFICATION)

        scored_list.append({
            '歌曲標題': titles_map[music_id],
            'ID': m['music_id'],
            '難度': m['difficulty'],
            '等級': levels.get((music_id, diff), 0),
            '時間': m['music_time'],
            '活動PT': event_pt,
            '秒均分': (event_pt/(m['music_time'] + times)),
            '周回': 3600 / (m['music_time'] + times),
            })

    scored_list.sort(key=lambda x: x['活動PT'], reverse=True)

    efficiency_result = []
    min_duration = float('inf')

    for song in scored_list:
        if song['等級'] > maxdiff:
            continue
        if no_append and song['難度'] == 'append':
            continue
        if song['時間'] < min_duration or (song['ID'] == 613 and song['難度'] == 'master'):
            # song['上首秒均分差'] = (efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0
            # song['上首周回差'] = (efficiency_result[-1]['周回'] - song['周回']) * -1 if efficiency_result else 0
            # song['增長率'] = ((efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0 ) / ((efficiency_result[-1]['時間'] if efficiency_result else 1) - song['時間'])
            # song['每小時分數'] = best_pt(song)
            song['五體每小時分數'] = (song['秒均分']) * 3600 * 25
            efficiency_result.append(song)
            min_duration = song['時間']
    return pd.DataFrame(efficiency_result)

if __name__ == "__main__":
    df_efficiency = calculate_efficiency()
    print("\n世界計畫活動效率曲清單 (Pareto Frontier)：\n")
    display(df_efficiency)

#時間體力固定 計算最佳分數

In [ ]:
# --- 設定與常數 ---
USER_POWER = int(input("綜合力"))
SKILL_MAGNIFICATION = round(sum(map(float,input("技能以空白鍵分開").split(" "))) / 5,2)
s6 = float(input("s6"))
maxdiff = int(input("最高難度(lv)"))
bonus = float(input("活動加成")) / 100
times = 3600 / float(input("周回")) - 74.8
energy_per_hour = float(input("每小時消耗體力"))
# no_append = int(input("0無1有"))


# USER_POWER = 350000
# SKILL_MAGNIFICATION = 2.4
# s6 = 2.5
# maxdiff = 38
# bonus = 395 / 100
# times = 3600 / 28 - 74.8
# energy_per_hour = 100
no_append = False

DIFFICULTY_RANK = {
    'easy': 1, 'normal': 2, 'hard': 3,
    'expert': 4, 'master': 5, 'append': 6
}


def calculate_event_pt(meta, power, skill_mag):
    eff_multiplier = (
        (meta['base_score'] * 1) +
        (meta['fever_score'] * 0.5) +
        (sum(meta['skill_score_multi'][0:5]) * (skill_mag) ) +
        (meta['skill_score_multi'][5] * s6)
    )

    score_part = math.floor((eff_multiplier * power * 4 + 0.075 * power * 5) / 17000)

    event_pt = math.floor((score_part + 123) * (meta['event_rate'] / 100) * (1 + bonus))

    return event_pt

def get_levels_map():

    if not diff: return {}
    return {(d['musicId'], d['musicDifficulty']): d['playLevel'] for d in diff}

def best_pt(song):
    use_energy = min(energy_per_hour,song['周回'] * 10)
    pt = 0

    if use_energy >= song['周回'] * 5:
        pt += song['活動PT'] * song['周回'] * 5 * 5
        use_energy -= song['周回'] * 5
        pt += song['活動PT'] * use_energy * 2
    else:
        pt += use_energy * song['活動PT'] * 5
        pt += song['活動PT'] * (song['周回'] - (use_energy / 5))

    return pt

def calculate_efficiency():
    levels = get_levels_map()
    metas = resp.json()

    metas.sort(key=lambda x: (x['music_id'], DIFFICULTY_RANK.get(x['difficulty'], 0)))

    scored_list = []
    for m in metas:
        music_id = m['music_id']
        diff = m['difficulty']
        if music_id not in titles_map:
            continue

        event_pt = calculate_event_pt(m, USER_POWER, SKILL_MAGNIFICATION)

        scored_list.append({
            '歌曲標題': titles_map[music_id],
            '難度': m['difficulty'],
            '等級': levels.get((music_id, diff), 0),
            '時間': m['music_time'],
            '活動PT': event_pt,
            '秒均分': (event_pt/(m['music_time'] + times)),
            '周回': 3600 / (m['music_time'] + times),
            })

    scored_list.sort(key=lambda x: x['活動PT'], reverse=True)

    efficiency_result = []
    min_duration = float('inf')

    for song in scored_list:
        if song['等級'] > maxdiff:
            continue
        if no_append and song['難度'] == 'append':
            continue
        if song['時間'] < min_duration:
            # song['上首秒均分差'] = (efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0
            # song['上首周回差'] = (efficiency_result[-1]['周回'] - song['周回']) * -1 if efficiency_result else 0
            # song['增長率'] = ((efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0 ) / ((efficiency_result[-1]['時間'] if efficiency_result else 1) - song['時間'])
            song['五體每小時分數'] = (song['秒均分']) * 3600 * 25
            song['同資源每小時最高分數'] = best_pt(song)
            efficiency_result.append(song)
            min_duration = song['時間']

    efficiency_result.sort(key=lambda x: x['同資源每小時最高分數'], reverse=True)
    return pd.DataFrame(efficiency_result)

# --- 執行與顯示 ---
if __name__ == "__main__":
    df_efficiency = calculate_efficiency()

    print("\n世界計畫活動效率曲清單 (Pareto Frontier)：\n")
    display(df_efficiency)

綜合力350000
技能以空白鍵分開2 2 2 2 2
s62
最高難度(lv)38
活動加成435
周回28
每小時消耗體力220

世界計畫活動效率曲清單 (Pareto Frontier)：



,歌曲標題,難度,等級,時間,活動PT,秒均分,周回,五體每小時分數,同資源每小時最高分數
0,群青讃歌,append,29,99.0,1837,12.024500,23.564616,1.082205e+06,1.457603e+06
1,ビバハピ,master,29,94.0,1797,12.160673,24.361949,1.094461e+06,1.447356e+06
2,Polar Star,master,31,96.4,1803,12.006279,23.972603,1.080565e+06,1.441659e+06
3,ジャックポットサッドガール,master,29,83.5,1713,12.478926,26.225414,1.123103e+06,1.427582e+06
4,独りんぼエンヴィー,expert,22,74.8,1658,12.895556,28.000000,1.160600e+06,1.425880e+06
5,悪ノ娘,master,27,91.5,1741,11.984463,24.781198,1.078602e+06,1.413201e+06
6,ODDS＆ENDS,master,29,93.8,1746,11.831559,24.394966,1.064840e+06,1.407144e+06
7,幽光、1/fのゆらめき,expert,25,91.2,1718,11.850611,24.832479,1.066555e+06,1.395853e+06
8,サンドリヨン 10th Anniversary,master,29,117.6,1893,11.046182,21.007002,9.941564e+05,1.391819e+06
9,地球最後の告白を,append,32,128.0,1907,10.491198,19.805093,9.442078e+05,1.321891e+06


#時間分數固定 計算最低體力

In [ ]:
# --- 設定與常數 ---
USER_POWER = int(input("綜合力"))
SKILL_MAGNIFICATION = round(sum(map(float,input("技能以空白鍵分開").split(" "))) / 5,2)
s6 = float(input("s6"))
maxdiff = int(input("最高難度(lv)"))
bonus = float(input("活動加成")) / 100
times = 3600 / float(input("周回")) - 74.8
# energy_per_hour = float(input("每小時消耗體力"))
target_pt_per_hour = float(input("每小時目標pt數"))
# no_append = int(input("0無1有"))

# USER_POWER = 390000
# SKILL_MAGNIFICATION = 1.8
# s6 = 2
# maxdiff = 38
# bonus = 395 / 100
# times = 3600 / 28 - 74.8
# energy_per_hour = 100
# target_pt_per_hour = 1000000
no_append = False

DIFFICULTY_RANK = {
    'easy': 1, 'normal': 2, 'hard': 3,
    'expert': 4, 'master': 5, 'append': 6
}


def calculate_event_pt(meta, power, skill_mag):
    eff_multiplier = (
        (meta['base_score'] * 1) +
        (meta['fever_score'] * 0.5) +
        (sum(meta['skill_score_multi'][0:5]) * (skill_mag) ) +
        (meta['skill_score_multi'][5] * s6)
    )

    score_part = math.floor((eff_multiplier * power * 4 + 0.075 * power * 5) / 17000)

    event_pt = math.floor((score_part + 123) * (meta['event_rate'] / 100) * (1 + bonus))

    return event_pt

def get_levels_map():

    if not diff: return {}
    return {(d['musicId'], d['musicDifficulty']): d['playLevel'] for d in diff}

def best_pt(song):
    use_energy = min(energy_per_hour,song['周回'] * 10)
    pt = 0

    if use_energy >= song['周回'] * 5:
        pt += song['活動PT'] * song['周回'] * 5 * 5
        use_energy -= song['周回'] * 5
        pt += song['活動PT'] * use_energy * 2
    else:
        pt += use_energy * song['活動PT'] * 5
        pt += song['活動PT'] * (song['周回'] - (use_energy / 5))

    return pt

def min_energy(song):
    if song['秒均分'] * 3600 * 35 < target_pt_per_hour:
        return False

    if target_pt_per_hour <= song['秒均分'] * 3600 * 25:
        return (target_pt_per_hour - song['秒均分'] * 3600) / (song['活動PT'] * 24 / 5)

    if target_pt_per_hour <= song['秒均分'] * 3600 * 35:
        return (target_pt_per_hour - song['五體每小時分數']) / (song['活動PT'] * 2) + song['周回'] * 5
    # 4 5 5 5 5 2 2 2 2 2

def calculate_efficiency():
    levels = get_levels_map()
    metas = resp.json()

    metas.sort(key=lambda x: (x['music_id'], DIFFICULTY_RANK.get(x['difficulty'], 0)))

    scored_list = []
    for m in metas:
        music_id = m['music_id']
        diff = m['difficulty']
        if music_id not in titles_map:
            continue
        event_pt = calculate_event_pt(m, USER_POWER, SKILL_MAGNIFICATION)

        scored_list.append({
            'id': music_id,
            '歌曲標題': titles_map[music_id],
            '難度': m['difficulty'],
            '等級': levels.get((music_id, diff), 0),
            '時間': m['music_time'],
            '活動PT': event_pt,
            '秒均分': (event_pt/(m['music_time'] + times)),
            '周回': 3600 / (m['music_time'] + times),
            })

    scored_list.sort(key=lambda x: x['活動PT'], reverse=True)

    efficiency_result = []
    min_duration = float('inf')

    for song in scored_list:
        if song['等級'] > maxdiff:
            continue
        if no_append and song['難度'] == 'append':
            continue
        if song['時間'] < min_duration :
            # song['上首秒均分差'] = (efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0
            # song['上首周回差'] = (efficiency_result[-1]['周回'] - song['周回']) * -1 if efficiency_result else 0
            # song['增長率'] = ((efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0 ) / ((efficiency_result[-1]['時間'] if efficiency_result else 1) - song['時間'])
            song['五體每小時分數'] = (song['秒均分']) * 3600 * 25
            # song['同資源每小時分數'] = best_pt(song)
            song['最低體力需求'] = min_energy(song)
            efficiency_result.append(song)
            min_duration = song['時間']

    efficiency_result.sort(key=lambda x: x['最低體力需求'])

    return pd.DataFrame(efficiency_result)

# --- 執行與顯示 ---
if __name__ == "__main__":
    df_efficiency = calculate_efficiency()

    print("\n世界計畫活動效率曲清單 (Pareto Frontier)：\n")
    display(df_efficiency)

綜合力390000
技能以空白鍵分開2.4 2.4 2.4 2.4 2.4
s62.4
最高難度(lv)38
活動加成)435
周回28
每小時目標pt數1300000

世界計畫活動效率曲清單 (Pareto Frontier)：



,id,歌曲標題,難度,等級,時間,活動PT,秒均分,周回,五體每小時分數,最低體力需求
0,691,0.0000034,append,37,154.3,2186,10.506008,17.301751,9.455407e+05,167.583621
1,448,Sage,expert,29,150.7,2173,10.627402,17.606372,9.564661e+05,167.077844
2,154,地球最後の告白を,append,32,128.0,2141,11.778529,19.805093,1.060068e+06,155.058255
3,104,サンドリヨン 10th Anniversary,master,29,117.6,2139,12.481661,21.007002,1.123349e+06,146.327800
4,141,群青讃歌,append,29,99.0,2077,13.595474,23.564616,1.223593e+06,136.216755
5,610,Polar Star,master,31,96.4,2035,13.551180,23.972603,1.219606e+06,139.615799
6,11,ビバハピ,master,29,94.0,2030,13.737432,24.361949,1.236369e+06,137.482427
7,190,悪ノ娘,master,27,91.5,1966,13.533287,24.781198,1.217996e+06,144.761566
8,427,幽光、1/fのゆらめき,expert,25,91.2,1932,13.326764,24.832479,1.199409e+06,150.195329
9,382,心拍ペアリング,master,28,87.3,1930,13.681013,25.518987,1.231291e+06,145.395160


#分數體力固定 計算最短時間

In [ ]:
# --- 設定與常數 ---
USER_POWER = int(input("綜合力"))
SKILL_MAGNIFICATION = round(sum(map(float,input("技能以空白鍵分開").split(" "))) / 5,2)
s6 = float(input("s6"))
maxdiff = int(input("最高難度(lv)"))
bonus = float(input("活動加成")) / 100
times = 3600 / float(input("周回")) - 74.8
# energy_per_hour = float(input("每小時消耗體力"))
# target_pt_per_hour = float(input("每小時目標pt數"))
target_pt = float(input("目標pt數"))
have_energy = float(input("擁有體力數"))
# no_append = int(input("0無1有"))

# USER_POWER = 355000
# SKILL_MAGNIFICATION = 2.4
# s6 = 2.5
# maxdiff = 38
# bonus = 395 / 100
# times = 3600 / 27 - 74.8

# energy_per_hour = 1000

# target_pt_per_hour = 1000000

# target_pt = 120000000
# have_energy = 100000

no_append = False

DIFFICULTY_RANK = {
    'easy': 1, 'normal': 2, 'hard': 3,
    'expert': 4, 'master': 5, 'append': 6
}


def calculate_event_pt(meta, power, skill_mag):
    eff_multiplier = (
        (meta['base_score'] * 1) +
        (meta['fever_score'] * 0.5) +
        (sum(meta['skill_score_multi'][0:5]) * (skill_mag) ) +
        (meta['skill_score_multi'][5] * s6)
    )

    score_part = math.floor((eff_multiplier * power * 4 + 0.075 * power * 5) / 17000)

    event_pt = math.floor((score_part + 123) * (meta['event_rate'] / 100) * (1 + bonus))

    return event_pt

def get_levels_map():

    if not diff: return {}
    return {(d['musicId'], d['musicDifficulty']): d['playLevel'] for d in diff}

def max_pt(song):
    use_energy = min(energy_per_hour,song['周回'] * 10)
    pt = 0

    if use_energy >= song['周回'] * 5:
        pt += song['活動PT'] * song['周回'] * 5 * 5
        use_energy -= song['周回'] * 5
        pt += song['活動PT'] * use_energy * 2
    else:
        pt += use_energy * song['活動PT'] * 5
        pt += song['活動PT'] * (song['周回'] - (use_energy / 5))

    return pt

def min_energy(song):
    if song['秒均分'] * 3600 * 35 < target_pt_per_hour:
        return False

    if target_pt_per_hour <= song['秒均分'] * 3600 * 25:
        return (target_pt_per_hour - song['秒均分'] * 3600) / (song['活動PT'] * 24 / 5)

    if target_pt_per_hour <= song['秒均分'] * 3600 * 35:
        return (target_pt_per_hour - song['五體每小時分數']) / (song['活動PT'] * 2) + song['周回'] * 5

def min_time(song):
    games = 0

    if target_pt <= song['活動PT'] * 35 * have_energy / 10:
        games += target_pt / (song['活動PT'] * 35)
        return games * (song['時間'] + times)

    if target_pt <= song['活動PT'] * 25 * have_energy / 5:
        games += target_pt / (song['活動PT'] * 25)
        return games * (song['時間'] + times)

    if target_pt > song['活動PT'] * 25 * have_energy / 5:
        games += have_energy / 5
        games += (target_pt - song['活動PT'] * 25 * have_energy / 5) / (song['活動PT'])
        return games * (song['時間'] + times)


def calculate_efficiency():
    levels = get_levels_map()
    metas = resp.json()

    metas.sort(key=lambda x: (x['music_id'], DIFFICULTY_RANK.get(x['difficulty'], 0)))

    scored_list = []
    for m in metas:
        music_id = m['music_id']
        diff = m['difficulty']
        if music_id not in titles_map:
            continue

        event_pt = calculate_event_pt(m, USER_POWER, SKILL_MAGNIFICATION)

        scored_list.append({
            'id': music_id,
            '歌曲標題': titles_map[music_id],
            '難度': m['difficulty'],
            '等級': levels.get((music_id, diff), 0),
            '時間': m['music_time'],
            '活動PT': event_pt,
            '秒均分': (event_pt/(m['music_time'] + times)),
            '周回': 3600 / (m['music_time'] + times),
            })

    scored_list.sort(key=lambda x: x['活動PT'], reverse=True)

    efficiency_result = []
    min_duration = float('inf')

    for song in scored_list:
        if song['等級'] > maxdiff:
            continue
        if no_append and song['難度'] == 'append':
            continue
        if song['時間'] < min_duration:
            # song['上首秒均分差'] = (efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0
            # song['上首周回差'] = (efficiency_result[-1]['周回'] - song['周回']) * -1 if efficiency_result else 0
            # song['增長率'] = ((efficiency_result[-1]['秒均分'] - song['秒均分']) * -1 if efficiency_result else 0 ) / ((efficiency_result[-1]['時間'] if efficiency_result else 1) - song['時間'])
            song['五體每小時分數'] = (song['秒均分']) * 3600 * 25
            # song['同資源每小時分數'] = max_pt(song)
            # song['最低體力需求'] = min_energy(song)
            song['最短時間需求(秒)'] = min_time(song)
            efficiency_result.append(song)
            min_duration = song['時間']

    efficiency_result.sort(key=lambda x: x['最短時間需求(秒)'])

    return pd.DataFrame(efficiency_result)

# --- 執行與顯示 ---
if __name__ == "__main__":
    df_efficiency = calculate_efficiency()

    print("\n世界計畫活動效率曲清單 (Pareto Frontier)：\n")
    display(df_efficiency)

綜合力350000
技能以空白鍵分開2.3 2.3 2.3 2.3 2.3
s62.5
最高難度(lv)38
活動加成)435
周回28
目標pt數80000000
擁有體力數12000

世界計畫活動效率曲清單 (Pareto Frontier)：



,id,歌曲標題,難度,等級,時間,活動PT,秒均分,周回,五體每小時分數,最短時間需求(秒)
0,427,幽光、1/fのゆらめき,expert,25,91.2,1791,12.354158,24.832479,1.111874e+06,259022.094600
1,190,悪ノ娘,master,27,91.5,1820,12.528272,24.781198,1.127544e+06,255422.291994
2,610,Polar Star,master,31,96.4,1882,12.532344,23.972603,1.127911e+06,255339.304691
3,382,心拍ペアリング,master,28,87.3,1786,12.660253,25.518987,1.139423e+06,252759.558471
4,11,ビバハピ,master,29,94.0,1877,12.702049,24.361949,1.143184e+06,251927.848390
5,62,ジャックポットサッドガール,master,29,83.5,1785,13.003434,26.225414,1.170309e+06,246088.835534
6,74,独りんぼエンヴィー,expert,22,74.8,1738,13.517778,28.000000,1.216600e+06,236725.300016
7,691,0.0000034,append,37,154.3,2029,9.751459,17.301751,8.776313e+05,234397.159554
8,448,Sage,expert,29,150.7,2023,9.893803,17.606372,8.904423e+05,231024.846914
9,154,地球最後の告白を,append,32,128.0,1981,10.898302,19.805093,9.808472e+05,209731.222120


#全歌曲難度按pt排

In [ ]:
# --- 設定與常數 ---
USER_POWER = int(input("綜合力"))
SKILL_MAGNIFICATION = round(sum(map(float,input("技能以空白鍵分開").split(" "))) / 5,2)
s6 = float(input("s6"))
maxdiff = int(input("最高難度(lv)"))
bonus = float(input("活動加成")) / 100
times = 3600 / float(input("周回")) - 74.8
# energy_per_hour = float(input("每小時消耗體力"))
# no_append = int(input("append(0無1有)"))
# USER_POWER = 300000
# SKILL_MAGNIFICATION = 2.5
# s6 = 2.5
# maxdiff = 38
# bonus = 395 / 100
# times = 3600 / 28 - 74.8
energy_per_hour = 100
no_append = False

DIFFICULTY_RANK = {
    'easy': 1, 'normal': 2, 'hard': 3,
    'expert': 4, 'master': 5, 'append': 6
}

def calculate_event_pt(meta, power, skill_mag):
    eff_multiplier = (
        (meta['base_score'] * 1) +
        (meta['fever_score'] * 0.5) +
        (sum(meta['skill_score_multi'][0:5]) * (skill_mag) ) +
        (meta['skill_score_multi'][5] * s6)
    )

    score_part = math.floor((eff_multiplier * power * 4 + 0.075 * power * 5) / 17000)
    event_pt = math.floor((score_part + 123) * (meta['event_rate'] / 100) * (1 + bonus))

    return event_pt

def get_levels_map():

    if not diff: return {}
    return {(d['musicId'], d['musicDifficulty']): d['playLevel'] for d in diff}

def best_pt(song):
    use_energy = min(energy_per_hour,song['周回'] * 10)
    pt = 0

    if use_energy >= song['周回'] * 5:
        pt += song['活動PT'] * song['周回'] * 5 * 5
        use_energy -= song['周回'] * 5
        pt += song['活動PT'] * use_energy * 2
    else:
        pt += use_energy * song['活動PT'] * 5
        pt += song['活動PT'] * (song['周回'] - (use_energy / 5))

    return pt

def calculate_efficiency():
    levels = get_levels_map()
    metas = resp.json()

    metas.sort(key=lambda x: (x['music_id'], DIFFICULTY_RANK.get(x['difficulty'], 0)))

    scored_list = []
    for m in metas:
        music_id = m['music_id']
        diff = m['difficulty']
        if music_id not in titles_map:
            continue

        event_pt = calculate_event_pt(m, USER_POWER, SKILL_MAGNIFICATION)

        scored_list.append({
            '歌曲標題': titles_map[music_id],
            'ID': m['music_id'],
            '難度': m['difficulty'],
            '等級': levels.get((music_id, diff), 0),
            '時間': m['music_time'],
            '活動PT': event_pt,
            '秒均分': (event_pt/(m['music_time'] + times)),
            '周回': 3600 / (m['music_time'] + times),
            '時效': event_pt / m['music_time'],
            })

    scored_list.sort(key=lambda x: x['活動PT'], reverse=True)

    return pd.DataFrame(scored_list)

if __name__ == "__main__":
    df_efficiency = calculate_efficiency()
    print("\n世界計畫活動效率曲清單 (Pareto Frontier)：\n")
    display(df_efficiency)

導出google sheet

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_efficiency)